In [177]:
from __future__ import division

import numpy as np
from astropy.io import fits
from astropy.wcs import WCS

import os
import sys

In [178]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import rcParams
rcParams["font.size"] = 10
rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Computer Modern Sans"]
rcParams["text.usetex"] = True
rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, AnchoredText
from matplotlib.patches import Rectangle

In [179]:
home = os.getenv('HOME')

In [180]:
# Read in pointings information
names_header = ['dataset', 'targ_name', 'ra', 'dec', 'exp_time', 'inst', 'camera', 'filter']

pears_pt = np.genfromtxt(home + '/Desktop/FIGS/pears_pointings.txt', delimiter=',',
     dtype=None, names=names_header, skip_header=11)

In [181]:
# define FOV for ACS in degrees
acs_fov = 202/3600  # it is 202 arcseconds on a side

In [182]:
# define functions to convert sexagesimal to degrees
def get_ra_deg(ra_str):
    hr, ramin, rasec = np.asarray(ra_str.split(' ')).astype(np.float)
    ra = hr*15 + (ramin*15/60) + (rasec*15/3600)
    return ra

def get_dec_deg(dec_str):
    deg, arcmin, arcsec = np.asarray(dec_str.split(' ')).astype(np.float)
    if deg < 0.0:
        dec = deg - (arcmin/60) - (arcsec/3600)
    elif deg >= 0.0:
        dec = deg + (arcmin/60) + (arcsec/3600)
    return dec

In [183]:
# Convert the sexagesimal coordinates to degrees and then
# Separate RA and DEC for north and south
pears_ra = np.zeros(len(pears_pt))
pears_dec = np.zeros(len(pears_pt))

for i in range(len(pears_pt)):

    pears_ra[i] = get_ra_deg(pears_pt['ra'][i])
    pears_dec[i] = get_dec_deg(pears_pt['dec'][i])

pears_north_ind = np.where(pears_dec >= 0)[0]
pears_north_ra = pears_ra[pears_north_ind]
pears_north_dec = pears_dec[pears_north_ind]

pears_south_ind = np.where(pears_dec < 0)[0]
pears_south_ra = pears_ra[pears_south_ind]
pears_south_dec = pears_dec[pears_south_ind]

In [184]:
# read in dummy WCS
goodsn_full = fits.open(home + '/Desktop/FIGS/total_north_F814W.fits')
goodss_full = fits.open(home + '/Desktop/FIGS/GOODS-S.ACS.F814W.fits')
wcs_north = WCS(goodsn_full[0].header)
wcs_south = WCS(goodss_full[0].header)

In [189]:
# select only PEARS north prime ACS obits 
pears_acs_north_ind = np.where((pears_pt['inst'] == 'ACS') & (pears_dec >= 0) \
                                & (pears_pt['filter'] == 'G800L;CLEAR2L'))[0]

pears_north_ra_prime = np.zeros(len(pears_acs_north_ind))
pears_north_dec_prime = np.zeros(len(pears_acs_north_ind))
                                 
# convert to degrees again
for j in range(len(pears_pt['ra'][pears_acs_north_ind])):
    
    pears_north_ra_prime[j] = get_ra_deg(pears_pt['ra'][pears_acs_north_ind][j])
    pears_north_dec_prime[j] = get_dec_deg(pears_pt['dec'][pears_acs_north_ind][j])

In [190]:
# make figure
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection=wcs_north)

ax.set_xlabel('Right Ascension', labelpad=1)
ax.set_ylabel('Declination', labelpad=-0.2)

shp = goodsn_full[0].data.shape
ax.imshow(np.zeros(shp), origin='lower', cmap='Greys')

ax.set_autoscale_on(False)

# Need to get the orients right. Right now it doesn't show any orient.
for k in range(len(pears_north_ra_prime)):
    
    r = Rectangle((pears_north_ra_prime[k], pears_north_dec_prime[k]), \
                  acs_fov/np.cos(pears_north_dec_prime[k]*np.pi/180), acs_fov, \
                  edgecolor='red', lw=1, facecolor='none', transform=ax.get_transform('fk5'))
    # format in the line above is (ra, dec), width, height,...
    ax.add_patch(r)

ax.grid(True)
fig.savefig(home + '/Desktop/FIGS/pears_north_layout.eps', dpi=300, bbox_inches='tight')

plt.cla()
plt.clf()
plt.close()

In [191]:
##### SOUTH #####
# select only PEARS south prime ACS obits 
pears_acs_south_ind = np.where((pears_pt['inst'] == 'ACS') & (pears_dec < 0) \
                                & (pears_pt['filter'] == 'G800L;CLEAR2L'))[0]

pears_south_ra_prime = np.zeros(len(pears_acs_south_ind))
pears_south_dec_prime = np.zeros(len(pears_acs_south_ind))
                                 
# convert to degrees again
for j in range(len(pears_pt['ra'][pears_acs_south_ind])):
    
    pears_south_ra_prime[j] = get_ra_deg(pears_pt['ra'][pears_acs_south_ind][j])
    pears_south_dec_prime[j] = get_dec_deg(pears_pt['dec'][pears_acs_south_ind][j])

In [192]:
# make figure
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection=wcs_south)

ax.set_xlabel('Right Ascension', labelpad=1)
ax.set_ylabel('Declination', labelpad=-0.2)

shp = goodss_full[0].data.shape
ax.imshow(np.zeros(shp), origin='lower', cmap='Greys')

ax.set_autoscale_on(False)

# Need to get the orients right. Right now it doesn't show any orient.
for k in range(len(pears_south_ra_prime)):
    
    r = Rectangle((pears_south_ra_prime[k], pears_south_dec_prime[k]), \
                  acs_fov/np.cos(pears_south_dec_prime[k]*np.pi/180), acs_fov, \
                  edgecolor='red', lw=1, facecolor='none', transform=ax.get_transform('fk5'))
    # format in the line above is (ra, dec), width, height,...
    ax.add_patch(r)

ax.grid(True)
fig.savefig(home + '/Desktop/FIGS/pears_south_layout.eps', dpi=300, bbox_inches='tight')

plt.cla()
plt.clf()
plt.close()